<a href="https://colab.research.google.com/github/atstuyuki/mediapipe/blob/main/mediapipe_pose_movie_20230601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 動画ファイルからmediapipe_poseで関節位置の検出を行い、CSVに出力する

mediapipe のインストール

In [1]:
# ライブラリーのインストール
#!pip install numpy==1.19.3
!pip install mediapipe
# 初期設定 holistic とPoseを用意する　デフォルトの検出感度は0.5＞＞変更可能
import mediapipe as mp
import cv2
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.9)
pose= mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.9)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(color=[225,0,200],thickness=2, circle_radius=1) #color=[B,G,R]で指定,線の太さと〇の色も指定可能

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 41.1 MB/s eta 0:00:00


In [2]:
#手元の動画をgoogle colabにupload

from google.colab import files
import os
uploaded = files.upload()
#uploadedは辞書型なのでファイル名を取り出し、絶対パスを取得してfullpathという変数を当てる
for k in uploaded.keys():
  print('file name:',k)
fullpath=os.path.abspath(k)
print('fullpath:',fullpath)

Saving Movie_00_(【オリックス】山下舜平大選手ピッチングフォーム).mp4 to Movie_00_(【オリックス】山下舜平大選手ピッチングフォーム).mp4
file name: Movie_00_(【オリックス】山下舜平大選手ピッチングフォーム).mp4
fullpath: /content/Movie_00_(【オリックス】山下舜平大選手ピッチングフォーム).mp4


CSV保存するときのファイル名を入力しておく

In [3]:
movie_name= "data1"#@param {type:"string"}

# 動画を読み込んで画像サイズとフレームレートを表示

In [4]:
cap = cv2.VideoCapture(fullpath) #動画の読み込み
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #幅
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #高
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) #FPS
fourcc = cv2.VideoWriter_fourcc('m','p','4','v') #mp4出力
out = cv2.VideoWriter('out.mp4', fourcc, frame_rate, (width, height))
print('width:',width)
print('height:',height)
print('frame rate:',frame_rate)



width: 774
height: 510
frame rate: 30


各種測定関数の定義

In [5]:
#体軸からの肩の外転角度測定
#3次元座標の角度計算の定義
def calculate_3D_shoulder_angle(a,b,c,d,e):
    a = np.array(a) # 右肘
    b = np.array(b) # 右肩
    c = np.array(c) # 左肩
    #c1=[b[0], c[1],b[2]]#右肩のx, 左肩のy、右肩のzを取得
    d = np.array(d) # 右股
    e = np.array(e) # 左股
    #0がx座標　１がｙ座標　２がｚ座標
    #中点を定義
    global shoulder_mid
    shoulder_mid = (b+c)/2 #肩の中心座標
    global hip_mid
    hip_mid = (d+e)/2 #股関節の中心座標　world系では0になるはず
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=hip_mid-shoulder_mid #体軸のベクトル
    vec_b=a-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_b=np.linalg.norm(vec_b)
    inner_product=np.inner(vec_a, vec_b)
    cos=inner_product/(len_vec_a*len_vec_b)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#2次元からの角度計算の定義
def calculate_2D_shoulder_angle(a,b,c,d,e):
    a = np.array(a) # 右肘
    b = np.array(b) # 右肩
    c = np.array(c) # 左肩
    #c1=[b[0], c[1],b[2]]#右肩のx, 左肩のy、右肩のzを取得
    d = np.array(d) # 右股
    e = np.array(e) # 左股
    #0がx座標　１がｙ座標　２がｚ座標
    #中点を定義
    shoulder_mid = (b+c)/2 #肩の中心座標
    hip_mid = (d+e)/2 #股関節の中心座標　world系では0になるはず
  #[x,y,z]座標から[x,y]のみスライスしてベクトルを作成
    vec_a=hip_mid[0:2]-shoulder_mid[0:2]#体軸のベクトル
    vec_b=a[0:2]-b[0:2]
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_b=np.linalg.norm(vec_b)
    inner_product=np.inner(vec_a, vec_b)
    cos=inner_product/(len_vec_a*len_vec_b)
    
#角度（ラジアン）の計算
    rad=np.arccos(cos)
#ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#面積　sizeの計算 2Dの3点から外積を算出　
def calc_size(a, b, c):
    a=np.array(a[0:2])#先端の座標1(x,yのみ)
    b=np.array(b[0:2])#先端の座標2
    c=np.array(c[0:2])#基部の座標

    vec1=a-c
    vec2=b-c
    
    size=np.cross(vec1,vec2)*10#数字が小さいので10倍しておく
    size=np.abs(size)#絶対値をとる
    return size


#pointの距離計測の関数 a-b（2点間の座標）間の距離を基準となる距離(c-dの距離）で除したものとする
def point_distance(a,b,c,d):
    a=np.array(a)#遠位関節の座標
    b=np.array(b)#近位関節の座標
    c=np.array(c)#基準関節の座標1
    d=np.array(d)#基準関節の座標2
    
    dist1=np.linalg.norm(a-b)
    dist2=np.linalg.norm(c-d)
    distance=dist1/dist2
    return distance 

#3次元座標の角度計算の定義
def calculate_3D_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #0がx座標　１がｙ座標　２がｚ座標
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=a-b
    vec_c=c-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#2次元からの角度計算の定義
def calculate_2D_angle(a,b,c):
  a=np.array(a)#firtst
  b=np.array(b)#second
  c=np.array(c)#third
  #[x,y,z]座標から[x,y]のみスライスしてベクトルを作成
  vec_a=a[0:2]-b[0:2]
  vec_c=c[0:2]-b[0:2]
  len_vec_a=np.linalg.norm(vec_a)
  len_vec_c=np.linalg.norm(vec_c)
  inner_product=np.inner(vec_a, vec_c)
  cos=inner_product/(len_vec_a*len_vec_c)
    
#角度（ラジアン）の計算
  rad=np.arccos(cos)
#ラジアンから度数へ変換
  angle=np.rad2deg(rad)
        
  return angle


# サンプルビデオの読み込み＋静止画に分解

In [6]:
# video2images
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= fullpath,   
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 1#適宜調整
    length = 3500
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [7]:
#pose_landmarkのリストを取得して配列に変換

import numpy as np
landmarklist=[]#CSV用のheaderを作る
for index in range(len(mp_holistic.PoseLandmark)):
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_x')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_y')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_z')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_visibility')
#リストをarrayに変換
landmarklist=np.array(landmarklist).reshape(1,-1)
#landmarkshape の内容の確認
print('landmark list data type:',landmarklist.dtype)
print('landmark list:',landmarklist)
print('landmark list shape:',landmarklist.shape)


landmark list data type: <U27
landmark list: [['NOSE_x' 'NOSE_y' 'NOSE_z' 'NOSE_visibility' 'LEFT_EYE_INNER_x'
  'LEFT_EYE_INNER_y' 'LEFT_EYE_INNER_z' 'LEFT_EYE_INNER_visibility'
  'LEFT_EYE_x' 'LEFT_EYE_y' 'LEFT_EYE_z' 'LEFT_EYE_visibility'
  'LEFT_EYE_OUTER_x' 'LEFT_EYE_OUTER_y' 'LEFT_EYE_OUTER_z'
  'LEFT_EYE_OUTER_visibility' 'RIGHT_EYE_INNER_x' 'RIGHT_EYE_INNER_y'
  'RIGHT_EYE_INNER_z' 'RIGHT_EYE_INNER_visibility' 'RIGHT_EYE_x'
  'RIGHT_EYE_y' 'RIGHT_EYE_z' 'RIGHT_EYE_visibility' 'RIGHT_EYE_OUTER_x'
  'RIGHT_EYE_OUTER_y' 'RIGHT_EYE_OUTER_z' 'RIGHT_EYE_OUTER_visibility'
  'LEFT_EAR_x' 'LEFT_EAR_y' 'LEFT_EAR_z' 'LEFT_EAR_visibility'
  'RIGHT_EAR_x' 'RIGHT_EAR_y' 'RIGHT_EAR_z' 'RIGHT_EAR_visibility'
  'MOUTH_LEFT_x' 'MOUTH_LEFT_y' 'MOUTH_LEFT_z' 'MOUTH_LEFT_visibility'
  'MOUTH_RIGHT_x' 'MOUTH_RIGHT_y' 'MOUTH_RIGHT_z'
  'MOUTH_RIGHT_visibility' 'LEFT_SHOULDER_x' 'LEFT_SHOULDER_y'
  'LEFT_SHOULDER_z' 'LEFT_SHOULDER_visibility' 'RIGHT_SHOULDER_x'
  'RIGHT_SHOULDER_y' 'RIGHT_SHOULDER_z' 

# MediaPipeで静止画を処理

In [8]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
import time
import re
#変数の初期化

#timelist=[]
frame_num_list=['frame_number']#フレーム数を表示するリストを作るheaderを入れておく


data_land = np.zeros((0,132))

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./images/*.png')):
    files.append(name)

# Read images with OpenCV.openCVで画像データを読みこみ
images = {name: cv2.imread(name) for name in files}
w = width             # 画像の横幅を取得
h = height            # 画像の縦幅を取得

for name, image in images.items():
# Convert the BGR image to RGB and process it with MediaPipe Pose.
  if image is None:
      break
#画像のファイル名からフレーム番号を取得
  match = re.search(r'\d+', name)
  if match:
      frame_num = match.group()
#画像をmeidapipeで処理
  results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  landmarks = results.pose_landmarks
  if landmarks is None:
      print('検出できなかったフレーム番号:',name)
      continue  
  
  frame_num_list.append(frame_num)
  #timelist.append(time.time())


# Draw pose landmarks.描画はworld座標系を使わない
  annotated_image = image.copy()

  mp_drawing.draw_landmarks(
        image=annotated_image, 
        landmark_list=results.pose_landmarks, 
        connections=mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=drawing_spec,
        connection_drawing_spec=drawing_spec)
  
#get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換　
#CSVに保存するデータは掌の中央を原点とするworld座標系を使用する
  data_land2 = np.zeros((1,3))
    
  for x in range (0,33):#pose_landmarksかpose_world_landmarksか選ぶ
      data1 = results.pose_world_landmarks.landmark[x].x
      data2 = results.pose_world_landmarks.landmark[x].y
      data3 = results.pose_world_landmarks.landmark[x].z
      data4 = results.pose_world_landmarks.landmark[x].visibility
      keydata = np.hstack((data1,data2,data3,data4)).reshape(1,-1)
      data_land2 = np.hstack((data_land2,keydata))

  data_land2 = data_land2[:,3:]
  data_land = np.vstack((data_land,data_land2))

  final_data= np.vstack((landmarklist,data_land))
  
  # テキストの描画
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(annotated_image, frame_num, (10, 50), font, 0.5, (255, 0, 255), 1)
  cv2.imwrite(name, annotated_image)  

# 座標データとフレーム数情報を結合して最初に決めたfilenameでCSVに保存する
実行すると左のファイル一覧に　ファイル名.csvのcsvが作成される
右クリックでダウンロードする

In [9]:
frame_num_array=np.array(frame_num_list)
frame_num_array=np.reshape(frame_num_array,(len(frame_num_list),1))
frame_landmark=np.hstack((frame_num_array,final_data))
filename=movie_name+'.csv'
#savetxtではデータの型が合わないと保存できないことがあるのでfmtで文字の型を指定
np.savetxt(filename,frame_landmark,delimiter = ',',fmt = '%s')

# 処理した画像をmp4動画に変換
実行するとoutput.mp4といる動画が作成される　右クリックでダウンロード


In [10]:
# 既に output.mp4 があれば削除
import os
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [11]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

＊作成したmp4動画は、左側のファイル欄にある　./mediapipe/output.mp4 を右クリックするとダウンロードできます。